In [1]:
# !pip install ensemble-boxes
from ensemble_boxes import *
import pandas as pd
import os

In [9]:
first_df = pd.read_csv('./ensemble_iou_0.55_skip_0.01.csv')
second_df = pd.read_csv('../EfficientDet/efficentdet.csv')

image_size = 1024
first_df.head()

,PredictionString,image_id
0,7 0.9995051622390747 603.1793823242188 516.561...,test/0000.jpg
1,5 0.9714235663414001 347.131591796875 251.0539...,test/0001.jpg
2,1 0.9598628878593445 293.5701904296875 337.972...,test/0002.jpg
3,9 0.9886714220046997 165.60089111328125 267.61...,test/0003.jpg
4,0 0.8930162787437439 428.5323486328125 410.046...,test/0004.jpg


In [11]:
prediction_strings = []
file_names = []

for i in range(len(second_df)):
    boxes_list = []
    scores_list = []
    labels_list = []
    
    first_box = []
    first_score = []
    first_label = []
    
    second_box = []
    second_score = []
    second_label = []
    
    box = []
    
    if type(first_df.loc[i]['PredictionString']) == float:
        cur_first_df = [0, 0, 0, 0, 0, 0]
    else:
        cur_first_df = first_df.loc[i]['PredictionString'].split()
    
    
    for j, _ in enumerate(cur_first_df):
        if j % 6 == 0:
            first_label.append(int(cur_first_df[j]))
        elif j % 6 == 1:
            first_score.append(float(cur_first_df[j]))
        else:
            box.append(float(cur_first_df[j]) / (image_size))
            if len(box) == 4:
                first_box.append(box)
                box = []
                
    if type(second_df.loc[i]['PredictionString']) == float:
        cur_first_df = [0, 0, 0, 0, 0, 0]      
    else:
        cur_second_df = second_df.loc[i]['PredictionString'].split()
    
    for j, _ in enumerate(cur_second_df):
        if j % 6 == 0:
            second_label.append(int(cur_second_df[j]))
        elif j % 6 == 1:
            second_score.append(float(cur_second_df[j]))
        else:
            box.append(float(cur_second_df[j]) / (image_size))
            if len(box) == 4:
                second_box.append(box)
                box = []
                
    boxes_list.append(first_box)
    boxes_list.append(second_box)
    scores_list.append(first_score)
    scores_list.append(second_score)
    labels_list.append(first_label)
    labels_list.append(second_label)
    
    iou_thr = 0.55
    skip_box_thr = 0.01
    sigma = 0.1
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes * image_size
    
    prediction_string = ''
    for j in range(len(scores)):
        prediction_string += str(int(labels[j])) + ' ' + str(scores[j]) + ' ' + str(boxes[j][0]) + ' ' + str(boxes[j][1]) + ' ' + str(boxes[j][2]) + ' ' + str(boxes[j][3]) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(first_df.loc[i]['image_id'])

        
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'ensemble_iou_{iou_thr}_skip_{skip_box_thr}.csv', index=None)
submission.head(7)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


,PredictionString,image_id
0,7 0.9623802304267883 598.4094848632812 517.334...,test/0000.jpg
1,3 0.5009316205978394 343.8243408203125 232.855...,test/0001.jpg
2,1 0.5788471698760986 79.85917663574219 284.851...,test/0002.jpg
3,9 0.8028486371040344 153.6845245361328 250.352...,test/0003.jpg
4,1 0.8046484589576721 212.17593383789062 290.56...,test/0004.jpg
5,2 0.908728837966919 354.3797607421875 219.6691...,test/0005.jpg
6,5 0.47736239433288574 356.92095947265625 450.9...,test/0006.jpg
